In [5]:
import SimpleITK as sitk
import numpy as np
import os

orig_mandible_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mirr_mandible_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_registered_mirrored.nii.gz"

output_combined = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\combined_mandible_mask.nii.gz"

orig = sitk.ReadImage(orig_mandible_path)
mirr = sitk.ReadImage(mirr_mandible_path)

orig_np = sitk.GetArrayFromImage(orig)
mirr_np = sitk.GetArrayFromImage(mirr)

print("Original mandible loaded:", orig.GetSize())
print("Mirrored mandible loaded:", mirr.GetSize())

combined_np = np.logical_or(orig_np > 0, mirr_np > 0).astype(np.uint8)

combined = sitk.GetImageFromArray(combined_np)
combined.CopyInformation(orig)

sitk.WriteImage(combined, output_combined)

print("Combined mask saved to:", output_combined)


Original mandible loaded: (512, 512, 179)
Mirrored mandible loaded: (512, 512, 179)
Combined mask saved to: Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\combined_mandible_mask.nii.gz


In [6]:
import SimpleITK as sitk
import numpy as np
import os

combined_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\combined_mandible_mask.nii.gz"
output_midline_file = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\midline_x.txt"

combined = sitk.ReadImage(combined_path)
comb_np = sitk.GetArrayFromImage(combined)

coords = np.column_stack(np.nonzero(comb_np))  

print("Combined mask loaded.")
print("Nonzero voxels found:", len(coords))

physical_x = []
for idx in coords:
    pt = combined.TransformIndexToPhysicalPoint((int(idx[2]), int(idx[1]), int(idx[0])))
    physical_x.append(pt[0])

physical_x = np.array(physical_x)

leftmost = physical_x.min()
rightmost = physical_x.max()
mid_x = (leftmost + rightmost) / 2.0

with open(output_midline_file, "w") as f:
    f.write(str(mid_x))

print("Leftmost X:", leftmost)
print("Rightmost X:", rightmost)
print("MIDLINE X:", mid_x)
print("Saved midline to:", output_midline_file)


Combined mask loaded.
Nonzero voxels found: 45547
Leftmost X: -60.52887725830078
Rightmost X: 45.907623291015625
MIDLINE X: -7.310626983642578
Saved midline to: Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\midline_x.txt


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

orig_ct_path   = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
mirr_ct_path   = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz"

orig_mand_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mirr_mand_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_registered_mirrored.nii.gz"

midline_file = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\midline_x.txt"

orig_ct   = sitk.ReadImage(orig_ct_path)
mirr_ct   = sitk.ReadImage(mirr_ct_path)
orig_mand = sitk.ReadImage(orig_mand_path)
mirr_mand = sitk.ReadImage(mirr_mand_path)

with open(midline_file, "r") as f:
    mid_x = float(f.read())

print("Midline X =", mid_x)

orig_np = sitk.GetArrayFromImage(orig_ct)
mirr_np = sitk.GetArrayFromImage(mirr_ct)
orig_mand_np = sitk.GetArrayFromImage(orig_mand)
mirr_mand_np = sitk.GetArrayFromImage(mirr_mand)

plane = "Coronal"   
slice_index = 228  

def get_slice(img_np, m1_np, m2_np, plane, idx):
    if plane == "Axial":
        return img_np[idx], m1_np[idx], m2_np[idx]
    elif plane == "Coronal":
        return img_np[:, idx, :], m1_np[:, idx, :], m2_np[:, idx, :]
    elif plane == "Sagittal":
        return img_np[:, :, idx], m1_np[:, :, idx], m2_np[:, :, idx]
    else:
        raise ValueError("Plane must be axial/coronal/sagittal")

orig_slice, orig_mask_slice, mirr_mask_slice = get_slice(
    orig_np, orig_mand_np, mirr_mand_np, plane, slice_index
)

mirr_slice, _, _ = get_slice(
    mirr_np, orig_mand_np, mirr_mand_np, plane, slice_index
)
def compute_midline_pixel(img, plane, idx):
    size = img.GetSize()

    if plane == "Axial":
        row = size[1] // 2
        xs = [img.TransformIndexToPhysicalPoint((x, row, idx))[0] for x in range(size[0])]
        return np.argmin(np.abs(np.array(xs) - mid_x))

    elif plane == "Coronal":
        z = size[2] // 2
        xs = [img.TransformIndexToPhysicalPoint((x, idx, z))[0] for x in range(size[0])]
        return np.argmin(np.abs(np.array(xs) - mid_x))

    else:
        return None  

mid_col = compute_midline_pixel(orig_ct, plane, slice_index)

solid_red = np.zeros((*orig_mask_slice.shape, 4))
solid_red[..., 0] = 1      # R
solid_red[..., 3] = 0.5    # alpha only where mask > 0
solid_red[orig_mask_slice == 0] = 0

solid_blue = np.zeros((*orig_mask_slice.shape, 4))
solid_blue[..., 2] = 1     # B
solid_blue[..., 3] = 0.5   # alpha
solid_blue[mirr_mask_slice == 0] = 0

plt.figure(figsize=(10,10))

plt.imshow(orig_slice, cmap="gray", alpha=0.7)

plt.imshow(mirr_slice, cmap="bone", alpha=0.7)

plt.imshow(solid_red)

plt.imshow(solid_blue)

if mid_col is not None:
    plt.axvline(mid_col, color='yellow', linewidth=2)

orig_ct_patch = mpatches.Patch(
    facecolor='none', edgecolor='none', linewidth=0,
    label='Gray colourmap: Original CT'
)

mir_ct_patch = mpatches.Patch(
    facecolor='none', edgecolor='none', linewidth=0,
    label='Bone colourmap: Mirrored, registered CT'
)

orig_mand_patch = mpatches.Patch(color='red', label='Original Mandible Mask')
mir_mand_patch = mpatches.Patch(color='blue', label='Mirrored, Registered Mandible Mask')

plt.legend(
    handles=[orig_ct_patch, mir_ct_patch, orig_mand_patch, mir_mand_patch],
    loc='upper right'
)
plt.title(f"{plane} slice {slice_index} — all overlays")
plt.axis("off")
plt.show()


Midline X = -7.310626983642578


NameError: name 'img' is not defined

In [ ]:
import SimpleITK as sitk
import numpy as np
import os

orig_mandible_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mirr_mandible_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_registered_mirrored.nii.gz"
midline_file_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\midline_x.txt"

output_dir = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_splits"
os.makedirs(output_dir, exist_ok=True)

orig = sitk.ReadImage(orig_mandible_path)
mirr = sitk.ReadImage(mirr_mandible_path)

with open(midline_file_path, "r") as f:
    mid_x = float(f.read())

print("Midline X loaded:", mid_x)


def split_mask(mask_img, mid_x):
    """Split a mandible mask into left/right halves based on a midline X value in physical space."""
    
    arr = sitk.GetArrayFromImage(mask_img)
    coords = np.column_stack(np.nonzero(arr))  

    left_img  = sitk.Image(mask_img.GetSize(), sitk.sitkUInt8)
    right_img = sitk.Image(mask_img.GetSize(), sitk.sitkUInt8)
    left_img.CopyInformation(mask_img)
    right_img.CopyInformation(mask_img)

    for idx in coords:
       
        itk_idx = (int(idx[2]), int(idx[1]), int(idx[0]))
        phys_pt = mask_img.TransformIndexToPhysicalPoint(itk_idx)
        x = phys_pt[0]

        if x < mid_x:
            left_img.SetPixel(itk_idx, 1)
        else:
            right_img.SetPixel(itk_idx, 1)

    return left_img, right_img


print("Splitting original mandible...")
orig_left, orig_right = split_mask(orig, mid_x)

print("Splitting mirrored mandible...")
mirr_left, mirr_right = split_mask(mirr, mid_x)

sitk.WriteImage(orig_left,  os.path.join(output_dir, "original_left.nii.gz"))
sitk.WriteImage(orig_right, os.path.join(output_dir, "original_right.nii.gz"))
sitk.WriteImage(mirr_left,  os.path.join(output_dir, "mirrored_left.nii.gz"))
sitk.WriteImage(mirr_right, os.path.join(output_dir, "mirrored_right.nii.gz"))

print("\nSaved all splits to:", output_dir)


Midline X loaded: -7.310626983642578
Splitting original mandible...
Splitting mirrored mandible...

Saved all splits to: Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_splits
